# CORINE Landover Preprocessing Notebook (Vector)

Create a python envionment with the Version 3.10 and activate it i.E:
```
conda create -n corine python=3.10
conda activate corine
conda install -c conda-forge rasterio geopandas shapely gdal
```


load the border file using geopandas and check its CRS

## Python Imports

In [15]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

## File Paths

In [ ]:
# define vector data paths for swiss shape
border_gpkg_path = "../../../data/base/swissBOUNDARIES3D/swissBOUNDARIES3D_1_5_LV95_LN02.gpkg"

# define vector data paths for corine 
input_vector_path = "../../../data/preprocessing/corine/2018/U2018_CLC2018_V2020_20u1.gpkg"
output_vector_path = "../../../data/analysis/corine/2018/U2018_CLC2018_V2020_20u1.gpkg"

# define import 
input_vector_layer = "U2018_CLC2018_V2020_20u1"

## Define Swiss Shape to clip CORINE Vector Data

coordinate reprojection from wgs84 to lv95

In [17]:
# read swiss shape layer from geopackage
border_gdf = gpd.read_file(border_gpkg_path, layer="tlm_landesgebiet")

add 1000m buffer around swiss shape

In [18]:
# create a 1000m buffer around the swiss shape (without other countries in the dataset)
buffered_border_gdf = border_gdf[border_gdf["icc"] == "CH"].copy()
buffered_border_gdf["geometry"] = buffered_border_gdf.buffer(1000)

remove enclaves from swiss shape

In [19]:
def fill_all_holes(geom):
    return Polygon(geom.exterior) if geom.type == 'Polygon' else MultiPolygon([Polygon(g.exterior) for g in geom.geoms])

# fill empty areas from enlaves within the swiss shape
buffered_border_gdf["geometry"] = buffered_border_gdf.geometry.apply(fill_all_holes)

C:\Users\LeonardoS\AppData\Local\Temp\ipykernel_18524\3789408810.py:2: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  return Polygon(geom.exterior) if geom.type == 'Polygon' else MultiPolygon([Polygon(g.exterior) for g in geom.geoms])


## Clip CORINE Raster Data to Swiss Shape

coordinate reprojection from wgs84 to lv95

In [20]:
# read swiss shape layer from geopackage
corine_vector_gdf = gpd.read_file(
    input_vector_path,
    layer=input_vector_layer,
    bbox=(3926000, 2445000, 4467000, 2878000)
)

# reproject input data
corine_vector_gdf_2056 = corine_vector_gdf.to_crs(epsg=2056)

clip vector data to swiss shape

In [21]:
# remove all polygons which do intersect with the swiss shape
corine_vector_gdf_clipped = gpd.sjoin(corine_vector_gdf_2056, buffered_border_gdf, predicate="intersects", how="inner")
corine_vector_gdf_clipped = corine_vector_gdf_clipped[corine_vector_gdf_2056.columns]

save vector data with swiss shape

In [22]:
# save cliped vector data 
corine_vector_gdf_clipped.to_file(output_vector_path, driver="GPKG")